# Introduction

You are a Starbucks big data analyst ([that’s a real job!](https://www.forbes.com/sites/bernardmarr/2018/05/28/starbucks-using-big-data-analytics-and-artificial-intelligence-to-boost-performance/#130c7d765cdc)) looking to find the next store into a [Starbucks Reserve Roastery](https://www.businessinsider.com/starbucks-reserve-roastery-compared-regular-starbucks-2018-12#also-on-the-first-floor-was-the-main-coffee-bar-five-hourglass-like-units-hold-the-freshly-roasted-coffee-beans-that-are-used-in-each-order-the-selection-rotates-seasonally-5).  These roasteries are much larger than a typical Starbucks store and have several additional features, including various food and wine options, along with upscale lounge areas.  You'll investigate the demographics of various counties in the state of California, to determine potentially suitable locations.

<center>
<img src="https://i.imgur.com/BIyE6kR.png" width="450"><br/><br/>
</center>

Before you get started, run the code cell below to set everything up.

In [1]:
import math
import pandas as pd
import geopandas as gpd
#from geopy.geocoders import Nominatim            # What you'd normally run
from learntools.geospatial.tools import Nominatim # Just for this exercise

import folium 
from folium import Marker
from folium.plugins import MarkerCluster

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex4 import *

/opt/conda/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


You'll use the `embed_map()` function from the previous exercise to visualize your maps.

In [2]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

# Exercises

### 1) Geocode the missing locations.

Run the next code cell to create a DataFrame `starbucks` containing Starbucks locations in the state of California.

In [3]:
# Load and preview Starbucks locations in California
starbucks = pd.read_csv("../input/geospatial-learn-course-data/starbucks_locations.csv")
starbucks.head()

,Store Number,Store Name,Address,City,Longitude,Latitude
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79


In [4]:
starbucks.shape

(2821, 6)

In [5]:
starbucks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2821 entries, 0 to 2820
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store Number  2821 non-null   object 
 1   Store Name    2821 non-null   object 
 2   Address       2821 non-null   object 
 3   City          2821 non-null   object 
 4   Longitude     2816 non-null   float64
 5   Latitude      2816 non-null   float64
dtypes: float64(2), object(4)
memory usage: 132.4+ KB


In [6]:
starbucks.nunique()

Store Number    2821
Store Name      2821
Address         2806
City             530
Longitude        526
Latitude         489
dtype: int64

Most of the stores have known (latitude, longitude) locations.  But, all of the locations in the city of Berkeley are missing.

In [7]:
starbucks.isnull().sum()

Store Number    0
Store Name      0
Address         0
City            0
Longitude       5
Latitude        5
dtype: int64

In [8]:
starbucks.loc[starbucks.Longitude.isnull()==True]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [9]:
starbucks.loc[starbucks.Latitude.isnull()==True]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,NaN,NaN
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,NaN,NaN
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,NaN,NaN
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,NaN,NaN
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,NaN,NaN


In [10]:
# View rows with missing locations
rows_with_missing = starbucks[starbucks["City"]=="Berkeley"]

In [11]:
# Create the geocoder
geolocator = Nominatim(user_agent="kaggle_learn")

# Your code here
def my_geocoder(row):
    point = geolocator.geocode(row).point
    return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})

berkeley_locations = rows_with_missing.apply(lambda x: my_geocoder(x['Address']), axis=1)
starbucks.update(berkeley_locations)

# Check your answer
q_1.check()

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [12]:
starbucks[starbucks["City"]=="Berkeley"]

,Store Number,Store Name,Address,City,Longitude,Latitude
153,5406-945,2224 Shattuck - Berkeley,2224 Shattuck Avenue Berkeley CA,Berkeley,-122.268230,37.868839
154,570-512,Solano Ave,1799 Solano Avenue Berkeley CA,Berkeley,-122.280050,37.891383
155,17877-164526,Safeway - Berkeley #691,1444 Shattuck Place Berkeley CA,Berkeley,-122.269679,37.880907
156,19864-202264,Telegraph & Ashby,3001 Telegraph Avenue Berkeley CA,Berkeley,-122.259370,37.855947
157,9217-9253,2128 Oxford St.,2128 Oxford Street Berkeley CA,Berkeley,-122.266079,37.870410


In [13]:
# Line below will give you solution code
#q_1.solution()

### 2) View Berkeley locations.

Let's take a look at the locations you just found.  Visualize the (latitude, longitude) locations in Berkeley in the OpenStreetMap style. 

In [14]:
# Create a base map
m_2 = folium.Map(location=[37.88,-122.26], zoom_start=13)

# Your code here: Add a marker for each Berkeley location
for idx, row in starbucks[starbucks["City"]=='Berkeley'].iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(m_2)


# Show the map
embed_map(m_2, 'q_2.html')

Considering only the five locations in Berkeley, how many of the (latitude, longitude) locations seem potentially correct (are located in the correct city)?

In [15]:
# View the solution (Run this code cell to receive credit!)
q_2.b.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> All five locations appear to be correct!

### 3) Consolidate your data.

Run the code below to load a GeoDataFrame `CA_counties` containing the name, area (in square kilometers), and a unique id (in the "GEOID" column) for each county in the state of California.  The "geometry" column contains a polygon with county boundaries.

In [16]:
CA_counties = gpd.read_file("../input/geospatial-learn-course-data/CA_county_boundaries/CA_county_boundaries/CA_county_boundaries.shp")
CA_counties.head()

,GEOID,name,area_sqkm,geometry
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6..."
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7..."
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822..."
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3..."
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360..."


Next, we create three DataFrames:
- `CA_pop` contains an estimate of the population of each county.
- `CA_high_earners` contains the number of households with an income of at least $150,000 per year.
- `CA_median_age` contains the median age for each county.

In [17]:
CA_pop = pd.read_csv("../input/geospatial-learn-course-data/CA_county_population.csv", index_col="GEOID")
CA_high_earners = pd.read_csv("../input/geospatial-learn-course-data/CA_county_high_earners.csv", index_col="GEOID")
CA_median_age = pd.read_csv("../input/geospatial-learn-course-data/CA_county_median_age.csv", index_col="GEOID")

In [18]:
print(len(CA_pop))
print(len(CA_high_earners))
print(len(CA_median_age))
print(len(CA_counties))

58
58
58
58


In [19]:
#CA_high_earners

Use the next code cell to join the `CA_counties` GeoDataFrame with `CA_pop`, `CA_high_earners`, and `CA_median_age`.

Name the resultant GeoDataFrame `CA_stats`, and make sure it has 8 columns: "GEOID", "name", "area_sqkm", "geometry", "population", "high_earners", and "median_age".  Also, make sure the CRS is set to `{'init': 'epsg:4326'}`.

In [20]:
# Your code here
cols_to_add = CA_pop.join([CA_high_earners, CA_median_age]).reset_index()
CA_stats = CA_counties.merge(cols_to_add, on="GEOID")
CA_stats.crs = {'init': 'epsg:4326'}

# Check your answer
q_3.check()
CA_stats.head(10)

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0
6,6097,Sonoma County,4578.952090,"POLYGON ((-122.93507 38.31396, -122.93511 38.3...",499942,32713,41.4
7,6031,Kings County,3604.052342,"POLYGON ((-119.95894 36.25547, -119.95894 36.2...",151366,2943,31.5
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4
9,6061,Placer County,3890.821444,"POLYGON ((-121.06545 39.00654, -121.06538 39.0...",393149,28334,41.6


Now that we have all of the data in one place, it's much easier to calculate statistics that use a combination of columns.  Run the next code cell to create a "density" column with the population density.

In [21]:
CA_stats["density"] = CA_stats["population"] / CA_stats["area_sqkm"]
CA_stats.head()

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
0,6091,Sierra County,2491.995494,"POLYGON ((-120.65560 39.69357, -120.65554 39.6...",2987,111,55.0,1.198638
1,6067,Sacramento County,2575.258262,"POLYGON ((-121.18858 38.71431, -121.18732 38.7...",1540975,65768,35.9,598.376878
2,6083,Santa Barbara County,9813.817958,"MULTIPOLYGON (((-120.58191 34.09856, -120.5822...",446527,25231,33.7,45.499825
3,6009,Calaveras County,2685.626726,"POLYGON ((-120.63095 38.34111, -120.63058 38.3...",45602,2046,51.6,16.980022
4,6111,Ventura County,5719.321379,"MULTIPOLYGON (((-119.63631 33.27304, -119.6360...",850967,57121,37.5,148.788107


In [22]:
CA_stats.describe()

,GEOID,area_sqkm,population,high_earners,median_age,density
count,58.000000,58.000000,5.800000e+01,58.000000,58.000000,58.000000
mean,6058.000000,7309.790554,6.820180e+05,39038.482759,39.474138,157.229368
std,33.773757,8072.512258,1.475165e+06,82066.991389,6.473280,301.609797
min,6001.000000,600.588247,1.101000e+03,30.000000,30.600000,0.571993
25%,6029.500000,2535.404026,4.783625e+04,1317.500000,34.775000,8.867764
50%,6058.000000,4132.385962,1.862525e+05,6396.500000,37.600000,41.851351
75%,6086.500000,9926.991709,7.019488e+05,28001.500000,44.275000,127.438530
max,6115.000000,52071.981221,1.010552e+07,501413.000000,55.000000,1470.733077


In [23]:
# Your code here
sel_counties = CA_stats[((CA_stats.high_earners > 100000) &
                         (CA_stats.median_age < 38.5) &
                         (CA_stats.density > 285) &
                         ((CA_stats.median_age < 35.5) |
                         (CA_stats.density > 1400) |
                         (CA_stats.high_earners > 500000)))]


In [24]:
sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


### 5) How many stores did you identify?

When looking for the next Starbucks Reserve Roastery location, you'd like to consider all of the stores within the counties that you selected.  So, how many stores are within the selected counties?

To prepare to answer this question, run the next code cell to create a GeoDataFrame `starbucks_gdf` with all of the starbucks locations.

In [25]:
starbucks_gdf = gpd.GeoDataFrame(starbucks, geometry=gpd.points_from_xy(starbucks.Longitude, starbucks.Latitude))
starbucks_gdf.crs = {'init': 'epsg:4326'}

/opt/conda/lib/python3.7/site-packages/pyproj/crs/crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [26]:
starbucks_gdf

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry
0,10429-100710,Palmdale & Hwy 395,14136 US Hwy 395 Adelanto CA,Adelanto,-117.40,34.51,POINT (-117.40000 34.51000)
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000)
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000)
3,29839-255026,Target Anaheim T-0677,8148 E SANTA ANA CANYON ROAD AHAHEIM CA,AHAHEIM,-117.75,33.87,POINT (-117.75000 33.87000)
4,23463-230284,Safeway - Alameda 3281,2600 5th Street Alameda CA,Alameda,-122.28,37.79,POINT (-122.28000 37.79000)
...,...,...,...,...,...,...,...
2816,14071-108147,Hwy 20 & Tharp - Yuba City,"1615 Colusa Hwy, Ste 100 Yuba City CA",Yuba City,-121.64,39.14,POINT (-121.64000 39.14000)
2817,9974-98559,"Yucaipa & Hampton, Yucaipa","31364 Yucaipa Blvd., A Yucaipa CA",Yucaipa,-117.12,34.03,POINT (-117.12000 34.03000)
2818,79654-108478,Vons - Yucaipa #1796,33644 YUCAIPA BLVD YUCAIPA CA,YUCAIPA,-117.07,34.04,POINT (-117.07000 34.04000)
2819,6438-245084,Yucaipa & 6th,"34050 Yucaipa Blvd., 200 Yucaipa CA",Yucaipa,-117.06,34.03,POINT (-117.06000 34.03000)


In [27]:
sel_counties

,GEOID,name,area_sqkm,geometry,population,high_earners,median_age,density
5,6037,Los Angeles County,12305.376879,"MULTIPOLYGON (((-118.66761 33.47749, -118.6682...",10105518,501413,36.0,821.227834
8,6073,San Diego County,11721.342229,"POLYGON ((-117.43744 33.17953, -117.44955 33.1...",3343364,194676,35.4,285.237299
10,6075,San Francisco County,600.588247,"MULTIPOLYGON (((-122.60025 37.80249, -122.6123...",883305,114989,38.3,1470.733077


So, how many stores are in the counties you selected?

In [28]:
# Fill in your answer
locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
num_stores = len(locations_of_interest)

locations_of_interest.head(10)

,Store Number,Store Name,Address,City,Longitude,Latitude,geometry,index_right,GEOID,name,area_sqkm,population,high_earners,median_age,density
1,635-352,Kanan & Thousand Oaks,5827 Kanan Road Agoura CA,Agoura,-118.76,34.16,POINT (-118.76000 34.16000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
2,74510-27669,Vons-Agoura Hills #2001,5671 Kanan Rd. Agoura Hills CA,Agoura Hills,-118.76,34.15,POINT (-118.76000 34.15000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
14,76365-97162,Target Alhambra T-184,1220 West Main Street Alhambra CA,Alhambra,-118.14,34.09,POINT (-118.14000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
15,6794-41839,Fremont Ave & Mission Rd,"1131 S Fremont Ave, A Alhambra CA",Alhambra,-118.15,34.08,POINT (-118.15000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
16,11220-104633,"Atlantic & Valley, Alhambra",1410 South Atlantic Blvd. Alhambra CA,Alhambra,-118.13,34.08,POINT (-118.13000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
17,47671-258708,Albertsons - Alhambras 6543,2400 W. Commonwealth Alhambra CA,Alhambra,-118.15,34.09,POINT (-118.15000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
18,9385-96258,"Valley & Almansor, Alhambra",810 E VALLEY BLVD ALHAMBRA CA,ALHAMBRA,-118.11,34.08,POINT (-118.11000 34.08000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
19,5380-673,Main & 1st - Alhambra,101 W. Main Street Alhambra CA,Alhambra,-118.13,34.09,POINT (-118.13000 34.09000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
67,73672-96295,Albertsons-Arcadia #6561,298 E Live Oak Ave Arcadia CA,Arcadia,-118.02,34.11,POINT (-118.02000 34.11000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834
68,5497-1534,300 E. Huntington Dr. - Arcadia,"300 E Huntington Drive, B15, Santa Anita Arcad...",Arcadia,-118.02,34.14,POINT (-118.02000 34.14000),5,6037,Los Angeles County,12305.376879,10105518,501413,36.0,821.227834


### 6) Visualize the store locations.

Create a map that shows the locations of the stores that you identified in the previous question.

In [29]:
# Create a base map
m_6 = folium.Map(location=[37,-120], zoom_start=6)

# Your code here: show selected store locations
mc = MarkerCluster()

locations_of_interest = gpd.sjoin(starbucks_gdf, sel_counties)
for idx, row in locations_of_interest.iterrows():
    if not math.isnan(row['Longitude']) and not math.isnan(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))

m_6.add_child(mc)


# Show the map
embed_map(m_6, 'q_6.html')

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*